In [1]:
!pip install --upgrade --quiet pip 
!pip install -I --upgrade --quiet setuptools
!pip install --upgrade --quiet tensorflow edward pymongo

In [2]:
% matplotlib inline

In [3]:
%load_ext autoreload
%autoreload 2

In [68]:
import fetch.fetch as fetch
from edward.models import Normal
from modelling.models import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *

In [6]:
df = fetch.create_df(fetch.get_labelled_articles("209.177.92.45:80"))

In [69]:
v = vectorizer(df)
tfidf = create_tfidf(v, df)
word_count = create_word_count(df)

In [326]:
model = MultinomialNB(class_prior=[.9,.1])
# errors = add_predictions(model, df, cv = 10
# ans = [1 if l == 'accepted' else 0 for l in df.label]

prediction_probs = cross_val_predict(model, tfidf, df.label, cv=10, method = 'predict_proba')
predictions = cross_val_predict(model, tfidf, df.label, cv=10, method = 'predict')
# errors.shape
preds = ['accepted' if p[0] >= 0.5 else 'rejected' for p in prediction_probs]
# np.sum(np.round(preds) != np.array(ans))

print(np.sum(predictions != df.label))

print(roc_auc_score(df.label == 'accepted', predictions == 'accepted'))
print(precision_score(df.label == 'accepted', predictions == 'accepted'))
print(average_precision_score(df.label == 'accepted', [p[0] for p in prediction_probs]))

519
0.709406494961
0.717011995638
0.906888031411


In [70]:
df[df.label != 'accepted'].shape

(893, 10)

In [74]:
def evaluate(model, tfidf, df, method = 'predict_proba'):
    ans = [1 if l == 'accepted' else 0 for l in df.label]
    probs = cross_val_predict(model, tfidf, ans, cv=10, method=method)
    predictions = cross_val_predict(model, tfidf, ans, cv=10, method='predict')

    probs = probs if method == 'decision_function' else [p[1] for p in probs] 
                           
    print (average_precision_score(ans, probs), roc_auc_score(ans, probs, average = None))
    print (precision_score(ans, predictions), recall_score(ans, predictions), f1_score(ans, predictions))
    # precision, recall, _ = precision_recall_curve(ans, guess)
    fpr, tpr, _ = roc_curve(ans, probs)
    plt.step(fpr, tpr)
    # plt.step(precision, recall)
    return probs, ans, predictions

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
evaluate(model, tfidf.toarray(), df)

In [75]:
model = RandomForestClassifier(200, max_depth = 3)
p, a, _ = evaluate(model, word_count, df)

0.788921864084 0.747265533308
0.631553631554 1.0 0.774174528302


In [77]:
from sklearn import svm

model = svm.LinearSVC()
g,a,p = evaluate(model, word_count, df, 'decision_function')

0.913129243774 0.894070129339
0.738550724638 0.970297029703 0.838709677419


In [ ]:
model = svm.OneClassSVM()
g,a,p = evaluate(model, tfidf, df, 'decision_function')

In [76]:
model = MultinomialNB(alpha = .001, class_prior = [0.5,0.5])
g, ans, preds = evaluate(model, word_count, df)

0.912042278343 0.899900027526
0.842391304348 0.708301599391 0.76954902772
